# BigtableByteStore

This guide covers how to use Google Cloud Bigtable as a key-value store.

[Bigtable](https://cloud.google.com/bigtable) is a key-value and wide-column store, ideal for fast access to structured, semi-structured, or unstructured data. 

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googleapis/langchain-google-bigtable-python/blob/main/docs/key_value_store.ipynb)

## Overview

The `BigtableByteStore` is a key-value store implementation that uses Google Cloud Bigtable as the backend. It supports both synchronous and asynchronous operations for setting, getting, and deleting key-value pairs.

### Integration details
| Class | Package | Local | JS support | Package downloads | Package latest |
| :--- | :--- | :---: | :---: | :---: | :---: |
| [BigtableByteStore](https://github.com/googleapis/langchain-google-bigtable-python/blob/main/src/langchain_google_bigtable/key_value_store.py) | [langchain-google-bigtable](https://pypi.org/project/langchain-google-bigtable/) | ❌ | ❌ | ![PyPI - Downloads](https://img.shields.io/pypi/dm/langchain-google-bigtable?style=flat-square&label=%20) | ![PyPI - Version](https://img.shields.io/pypi/v/langchain-google-bigtable) |

## Setup

### Prerequisites

You need a Google Cloud project with the Bigtable API enabled, an active Bigtable instance, and a table. You can follow these guides for detailed instructions:
* [Create a Google Cloud Project](https://developers.google.com/workspace/guides/create-project)
* [Enable the Bigtable API](https://console.cloud.google.com/flows/enableapi?apiid=bigtable.googleapis.com)
* [Create a Bigtable instance and table](https://cloud.google.com/bigtable/docs/creating-instance)

### Installation

The integration lives in the `langchain-google-bigtable` package. The command below also installs `langchain-google-vertexai`, which is needed for the embedding cache example.

In [ ]:
%pip install -qU langchain-google-bigtable langchain-google-vertexai

## Authentication

Set your Google Cloud project ID and authenticate to your account.

In [ ]:
# @markdown Please fill in with your Google Cloud project ID, Bigtable instance ID, and table ID.
PROJECT_ID = "your-gcp-project-id"  # @param {type:"string"}
INSTANCE_ID = "your-instance-id"  # @param {type:"string"}
TABLE_ID = "your-table-id"  # @param {type:"string"}

!gcloud config set project {PROJECT_ID}

from google.colab import auth
auth.authenticate_user()

## Instantiation

To use `BigtableByteStore`, we first need to ensure a table exists and then initialize a `BigtableEngine` to manage connections.

### BigtableEngine
A `BigtableEngine` object handles the execution context of the store, particularly for asynchronous operations. It's highly recommended to initialize a single `BigtableEngine` instance and reuse it across multiple stores for better performance and resource management.

In [ ]:
from langchain_google_bigtable import (
    BigtableByteStore,
    BigtableEngine,
    init_key_value_store_table,
)

# Ensure the table and column family exist.
init_key_value_store_table(
    project_id=PROJECT_ID,
    instance_id=INSTANCE_ID,
    table_id=TABLE_ID,
)

# Initialize the engine to manage async operations.
engine = await BigtableEngine.async_initialize(project_id=PROJECT_ID, instance_id=INSTANCE_ID)

# Create the store asynchronously using the engine.
store = await BigtableByteStore.create(
    engine=engine, 
    table_id=TABLE_ID
)

## Usage

The `BigtableByteStore` supports both synchronous (e.g., `mset`, `mget`) and asynchronous (e.g., `amset`, `amget`) methods. This guide uses the async methods.

### Set
Use `amset` to save key-value pairs to the store.

In [ ]:
kv_pairs = [
    ("key1", b"value1"),
    ("key2", b"value2"),
    ("key3", b"value3"),
]

await store.amset(kv_pairs)

### Get
Use `amget` to retrieve values for a given list of keys. If a key is not found, `None` is returned for that key.

In [ ]:
retrieved_vals = await store.amget(["key1", "key2", "nonexistent_key"])
print(retrieved_vals)

### Delete
Use `amdelete` to remove keys from the store.

In [ ]:
await store.amdelete(["key3"])

# Verifying the key was deleted
await store.amget(["key1", "key3"])

### Iterate over keys
Use `ayield_keys` to iterate over all keys in the store or over keys with a specific prefix.

In [ ]:
all_keys = [key async for key in store.ayield_keys()]
print(f"All keys: {all_keys}")

prefixed_keys = [key async for key in store.ayield_keys(prefix="key1")]
print(f"Prefixed keys: {prefixed_keys}")

## Advanced Usage: Embedding Caching

A common use case for a key-value store is to cache the results of expensive operations, like computing text embeddings. This saves both time and money on subsequent requests for the same text.

In [ ]:
from langchain.embeddings import CacheBackedEmbeddings
from langchain_google_vertexai.embeddings import VertexAIEmbeddings

# Initialize the underlying embeddings model
underlying_embeddings = VertexAIEmbeddings(project=PROJECT_ID, model_name="textembedding-gecko@003")

# Create the cached embedder, using a namespace to avoid key collisions
cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings,
    store,
    namespace="text-embeddings"
)

print("First call (computes and caches embedding):")
%time embedding_result_1 = await cached_embedder.aembed_query("Hello, world!")

print("\nSecond call (retrieves from cache):")
%time embedding_result_2 = await cached_embedder.aembed_query("Hello, world!")

## API reference

For full details on the `BigtableByteStore` class, see the [API reference](https://python.langchain.com/v0.2/api_reference/google_bigtable/storage/langchain_google_bigtable.storage.BigtableByteStore.html).